In [1]:
import os
from IPython.display import Image as ipy_image
from IPython.display import display

#from utils.demo_util import get_shape_comp_model, get_shape_comp_opt, make_dummy_batch
#from utils.qual_util import load_bert2vqsc_model, get_lang_prob, save_mesh_as_gif
from utils.qual_util import get_lang_prob, save_mesh_as_gif
from utils.util import seed_everything
from utils.util_3d import init_mesh_renderer, sdf_to_mesh
import os
from termcolor import colored, cprint
import torch
import utils.util as util
from tqdm.notebook import tqdm
from datasets.text2shape import  Text2Shape


from datasets.text2shapepp_dataset import  Text2ShapePP


%reload_ext autoreload
%autoreload 2

In [2]:
class Opt():
    print("class")
    
def get_shape_comp_opt(gpu_id=0):
    opt = Opt()

    # args
    gpuid=[gpu_id]
    batch_size=16
    max_dataset_size=64

    name='test_New bert'

    # default args
    opt.serial_batches = False
    opt.nThreads = 4

    # important args
    opt.dataset_mode = 'shapenet_code'
    opt.seed = 111
    opt.isTrain = True
    opt.gpu_ids = gpuid
    opt.max_dataset_size=64
    opt.device = 'cuda:%s' % gpuid[0]
    opt.batch_size = batch_size
    opt.max_dataset_size = max_dataset_size
    opt.logs_dir="../logs"
    opt.name = name
    opt.lr = 1e-4
    opt.save_latest_freq = 1000
    opt.print_freq = 300
    #utils.util.seed_everything(opt.seed)
    opt.phase = 'test'
    opt.nepochs = 60
    opt.nepochs_decay = 5
    return opt
opt = get_shape_comp_opt()

class


In [31]:


dataset = Text2Shape()
#opt = Opt()

dataset.initialize(opt)

In [32]:
len(dataset)

64

In [7]:
for i in tqdm(range(len(dataset))):
    dataset[i]

  0%|          | 0/64 [00:00<?, ?it/s]

In [5]:
dataset[19]["current_text"]

'and rocking feet'

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

text1 = "thin legs"
text2 = "round arms"
# tokenized1 = tokenizer(text1,return_tensors='pt',padding=True)
# x = bertmodel(**tokenized1).pooler_output
# x.shape
# tokenized2 = tokenizer(text2,return_tensors='pt',padding=True)
# x2 = bertmodel(**tokenized2).pooler_output
# x2.shape
x1 = model.encode(text1)
x2 = model.encode(text2)
x1.shape
x2.shape
model.eval()



SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [147]:
cos = torch.nn.CosineSimilarity(dim=0)

In [148]:
cos(torch.Tensor(x1),torch.Tensor(x2))

tensor(0.4562)

In [6]:
from models.bert2vq_scmodel import BERT2VQSCModel

In [46]:
m = BERT2VQSCModel()
m.initialize(opt)

---------- Networks initialized -------------
-----------------------------------------------


In [122]:
# data = dataset[4]
# data2 = dataset[2]
# data["z_set_prev"] = data["z_set_prev"].unsqueeze(0)
# data["z_set_target"] = data["z_set_target"].unsqueeze(0)
# data["current_text"] = [data["current_text"]]


In [123]:
m.set_input(data)
m.forward()

In [125]:
m.backward()

In [34]:
x = dataset[0]["z_set_prev"].shape
y = dataset[3]["z_set_prev"].shape
x

torch.Size([8, 8, 8, 512])

In [129]:
x[x!=0].shape

torch.Size([2034])

In [35]:
from torch.utils.data import DataLoader
train_dl =  DataLoader(
            dataset,
            batch_size=opt.batch_size,
            shuffle=False,
            drop_last=True,
            num_workers=0)

In [63]:
options=opt
model = m
def train_one_epoch2(pt_profiler=None,epoch=0):
    global total_steps

    epoch_iter = 0
    for i, data in enumerate(train_dl), total=len(train_dl):
        iter_start_time = time.time()
        #visualizer.reset()
        #import pdb;pdb.set_trace()
        total_steps += options.batch_size
        epoch_iter += options.batch_size
        model.set_input(data)

        model.optimize_parameters(total_steps)

        nBatches_has_trained = total_steps // options.batch_size
        # if total_steps % options.print_freq == 0:
        import pdb;pdb.set_trace()
        if nBatches_has_trained % options.print_freq == 0:
            errors = model.get_current_errors()
            import pdb;pdb.set_trace()
            print(model.loss,"LOSS")
            t = (time.time() - iter_start_time) / options.batch_size
            visualizer.print_current_errors(
                epoch, epoch_iter, total_steps, errors, t)

        # if (nBatches_has_trained % options.display_freq == 0) or i == 0:
        #     # eval
        #     model.inference(data)
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='train')

        #     #model.set_input(next(test_dg))
        #     test_data = next(test_dg)
        #     model.inference(test_data.unsqueeze(0))
        #     visualizer.display_current_results(
        #        model.get_current_visuals(), total_steps, phase='test')

#         if total_steps % options.save_latest_freq == 0:
#             cprint('saving the latestmodel (epoch %d, total_steps %d)' %
#                    (epoch, total_steps), 'blue')
#             latest_name = f'epoch-latest'
#             model.save(latest_name)

        if pt_profiler is not None:
            pt_profiler.step()

SyntaxError: invalid syntax (2724499001.py, line 7)

In [65]:
import time
cprint('[*] Start training. name: %s' % options.name, 'blue')
total_steps = 0
for epoch in tqdm(range(options.nepochs + options.nepochs_decay)):
    epoch_start_time = time.time()
    # epoch_iter = 0

    # profile
    train_one_epoch2(None,epoch)

#     if epoch % 100000 == 0:
#         cprint('saving the model at the end of epoch %d, iters %d' %
#                (epoch, total_steps), 'blue')
#         latest_name = f'epoch-latest'
#         model.save(latest_name)
#         cur_name = f'epoch-{epoch}'
#         model.save(cur_name)

    cprint(f'[*] End of epoch %d / %d \t Time Taken: %d sec \n%s' %
           (
               epoch, options.nepochs + options.nepochs_decay,
               time.time() - epoch_start_time,
               os.path.abspath(os.path.join(options.logs_dir, options.name))
           ), 'blue', attrs=['bold']
           )
    model.update_learning_rate()


[*] Start training. name: test_New bert


  0%|          | 0/65 [00:00<?, ?it/s]

NameError: name 'train_one_epoch2' is not defined